In [3]:
import numpy as np
from math import *

In [237]:
class HiddenMarkov(object):
    def __init__(self,Q,V,A=None,B=None,PI=None,O=None):
        self.Q=np.array(Q)
        self.V=np.array(V)
        self.A=np.array(A)
        self.B=np.array(B)
        self.PI=np.array(PI)
        self.O=np.array(O)
        self.gama=None
        self.xi=None
    def forward(self,A=None,B=None,PI=None,O=None):

        n,T=len(self.PI),len(self.O)
        #创建前向概率矩阵，i行j列表示j时刻处于i状态且观测为O[:j]的概率
        alp=np.empty((n,T))
        #观测序列转化为索引
        b=np.where(self.V==self.O[0])[0][0]        
        alp[:,0]=self.PI*self.B[:,b]
        for i in range(1,T):
            b=np.where(self.V==self.O[i])[0][0] 
            for j in range(n):
                alp[j,i]=np.sum(alp[:,i-1]*self.A[:,j])*self.B[j,b]
        
        return alp
    def backward(self,A=None,B=None,PI=None,O=None):

        n,T=len(self.PI),len(self.O)
        #创建后向概率矩阵，i行j列表示j时刻处于i状态的前提下，观测为O[j+1:]的概率
        beta=np.empty((n,T))
        beta[:,-1]=1
        for i in range(T-2,-1,-1):
            b=np.where(self.V==self.O[i+1])[0][0] 
            for j in range(n):
                beta[j,i]=np.sum(self.A[j,:]*self.B[:,b]*beta[:,i+1])
        
        return beta
        
    def cal(self):
        alp=self.forward(self.A,self.B,self.PI,self.O)
        beta=self.backward(self.A,self.B,self.PI,self.O)
        n,T=np.shape(alp)
        self.gama=np.empty(np.shape(alp))
        self.xi=np.empty((T,n,n))
        for i in range(T):
            
            self.gama[:,i]=alp[:,i]*beta[:,i]/(sum(alp[:,i]*beta[:,i]))
            if i<T-1:
                b=np.where(self.V==self.O[i+1])[0][0]       
                #创建行列向量得到矩阵
                ax,bx=np.ix_(alp[:,i],beta[:,i+1]*self.B[:,b])
                #一维行列得到矩阵，然后矩阵点乘
                mat=ax.dot(bx)*self.A
                self.xi[i,:,:]=mat/np.sum(mat,axis=(0,1))

        return None
    def BaumWelch_train(self):
        self.cal()
        self.A=np.sum(self.xi[:-1,:,:],axis=0)/np.sum(self.gama[:,:-1],axis=1)
        for j in range(self.B.shape[1]):
            # 统计第j类状态在观测序列中出现的索引
            b=np.where(self.O==self.V[j])[0]
            if len(b)<2:
                self.B[:,j]=np.sum(self.gama[:,b])/np.sum(self.gama,axis=1)
            else:
                self.B[:,j]=np.sum(self.gama[:,b],axis=1)/np.sum(self.gama,axis=1)
        self.PI=self.gama[:,0]
        return None
    def BaumWelch(self,A,B,PI,O):
        self.A,self.B,self.PI,self.O=np.array(A),np.array(B),np.array(PI),np.array(O)
        epoch,i=100,1
        eps,EPS=1,1e-9
        norm=np.linalg.norm
        while eps>EPS and i<epoch:
            A,B,PI,O=self.A,self.B,self.PI,self.O
            self.BaumWelch_train()
            i=i+1            
            eps=norm(self.A-A)+norm(self.B-B)+norm(self.PI-PI)
        return self.A,self.B,self,PI
    
    def viterbi(self,A,B,PI,O):
        self.A,self.B,self.PI,self.O=np.array(A),np.array(B),np.array(PI),np.array(O)
        n,T=self.A.shape[0],self.O.shape[0]
        # 定义前项概率状态序列的最大值时刻矩阵，以及相应索引矩阵
        deta,psai=np.empty((n,T)),np.empty((n,T),dtype='int32')
        b=np.where(self.V==self.O[0])[0][0]
        deta[:,0]=self.PI*self.B[:,b]
        psai[:,0]=0
        for t in range(1,T):
            b=np.where(self.V==self.O[t])[0][0]
            for i in range(n):
                dji=deta[:,t-1]*self.A[:,i]
                deta[i,t]=np.max(dji)*self.B[i,b]
                psai[i,t]=np.argmax(dji)
        I=np.empty(T,dtype='int32')
        I[-1]=np.argmax(deta[:,-1])
        print(I[-1])
        for t in range(T-2,-1,-1):
            I[t]=psai[I[t+1],t+1]
        return I

In [238]:
Q = [1, 2, 3]
V = ['红', '白']
A = [[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]]
B = [[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]]
# O = ['红', '白', '红', '红', '白', '红', '白', '白']
O = ['红', '白', '红', '白']    #习题10.1的例子
PI = [0.2, 0.4, 0.4]
hm=HiddenMarkov(Q,V,A,B,PI,O)

In [239]:
O=['红','白','红']
hm.viterbi(A,B,PI,O)

2


array([2, 2, 2])

In [148]:
b=hm.backward()

In [149]:
c=np.where(hm.V==hm.O[0])[0][0]
print(np.sum(hm.PI*hm.B[:,c]*b[:,0]))
print(np.sum(a[:,-1]))

0.06009079999999999
0.06009079999999999


In [208]:
hm.BaumWelch(A,B,PI,O)

(array([[0.  , 0.  , 0.25],
        [0.  , 0.  , 0.25],
        [2.  , 2.  , 0.  ]]), array([[0., 1.],
        [0., 1.],
        [1., 0.]]), <__main__.HiddenMarkov at 0x1fe74606b00>, array([0., 0., 1.]))

In [205]:
a=np.linalg.norm
a([1,2,3])

3.7416573867739413

In [209]:
A=np.arange(9).reshape((3,3))
b=np.array([1,2,1])

In [210]:
b*A

array([[ 0,  2,  2],
       [ 3,  8,  5],
       [ 6, 14,  8]])

In [212]:
print(b,A)

[1 2 1] [[0 1 2]
 [3 4 5]
 [6 7 8]]


In [216]:
num,ind=np.max([5,8,3])

TypeError: 'numpy.int32' object is not iterable

In [217]:
np.argmax([5,8,3])

1